# SQD for ATP

## Setup

In [11]:
import matplotlib.pyplot as plt; plt.rcParams.update({"font.family": "serif"})
import numpy as np
import pickle

import openfermion as of

import qiskit
from qiskit import qasm2, qasm3
from qiskit_aer import AerSimulator
import qiskit_ibm_runtime
from qiskit_ibm_runtime import SamplerV2 as Sampler

In [16]:
ibm_computer: str = "ibm_torino"
qubits = [
    72,
    *list(range(76, 84 + 1)),
    91, 92, 93,
    *list(range(95, 103 + 1)),
    110, 111,
    # *list(range(2, 12 + 1)),
    # 16, 17, 18,
    # *list(range(22, 31 + 1)),
]
nshots: int = 50_000

nqubits = len(qubits)
nqubits

24

In [ ]:
# Skip this cell if not running on hardware.
service = qiskit_ibm_runtime.QiskitRuntimeService(name="NERSC-US")
computer = service.backend(ibm_computer)
sampler = Sampler(computer)

In [19]:
computer

<IBMBackend('ibm_torino')>

In [21]:
fragment = "atp_0_be1_f8"

all_iterations = [1, 2, 3, 4, 5, 10, 15, 20, 25, 30, 35, 40]


In [22]:
for adapt_iterations in all_iterations:
    print("On iteration:", adapt_iterations)
    circuit_dir =  "circuits"
    circuit_fname = f"{fragment}_{adapt_iterations:03d}_adaptiterations.qasm"
    circuit_path = f"{circuit_dir}/{fragment}/{circuit_fname}"

    circuit = qasm3.load(circuit_path)

    base = circuit.copy()
    base = qiskit.transpiler.passes.RemoveBarriers()(base)

    to_run = base.copy()
    to_run.measure_all()
    to_run = [to_run]

    to_run = qiskit.transpile(
        to_run,
        optimization_level=3,
        backend=computer,
        initial_layout=qubits,
        seed_transpiler=123
    )

    for c in to_run:
        print(c.count_ops())
    
    job = sampler.run(to_run, shots=nshots)
    all_counts_hardware = []
    res = job.result()

    for r in res:
        all_counts_hardware.append(r.data.meas.get_counts())
    
    import datetime

    time_key = datetime.datetime.now().strftime("%Y_%m_%d_%H-%M-%S")
    fname = circuit_fname + f"_counts_{computer.name}_nshots_{nshots}_{time_key}"
    print("Saving bitstrings to", fname)
    pickle.dump(all_counts_hardware[0], open(fname, "wb"))

On iteration: 1
OrderedDict([('sx', 73), ('rz', 56), ('cz', 36), ('measure', 24), ('x', 13), ('barrier', 1)])
Saving bitstrings to atp_0_be1_f8_001_adaptiterations.qasm_counts_ibm_torino_nshots_50000_2026_02_14_13-42-34
On iteration: 2
OrderedDict([('sx', 309), ('rz', 223), ('cz', 160), ('measure', 24), ('x', 12), ('barrier', 1)])
Saving bitstrings to atp_0_be1_f8_002_adaptiterations.qasm_counts_ibm_torino_nshots_50000_2026_02_14_13-42-54
On iteration: 3
OrderedDict([('sx', 465), ('rz', 312), ('cz', 254), ('measure', 24), ('x', 20), ('barrier', 1)])
Saving bitstrings to atp_0_be1_f8_003_adaptiterations.qasm_counts_ibm_torino_nshots_50000_2026_02_14_13-43-14
On iteration: 4
OrderedDict([('sx', 742), ('rz', 517), ('cz', 400), ('x', 29), ('measure', 24), ('barrier', 1)])
Saving bitstrings to atp_0_be1_f8_004_adaptiterations.qasm_counts_ibm_torino_nshots_50000_2026_02_14_13-43-35
On iteration: 5
OrderedDict([('sx', 623), ('rz', 487), ('cz', 356), ('x', 32), ('measure', 24), ('barrier', 1)]

IBMNotAuthorizedError: '401 Client Error: Unauthorized for url: https://quantum.cloud.ibm.com/api/v1/jobs/d68ca8be4kfs73d2v0b0?exclude_params=true. {"errors":[{"code":1219,"message":"Error authenticating user.","solution":"Your IBM Quantum API token might be incorrect, expired, or revoked.\\nLog in to IBM Quantum Platform website and navigate to your account settings to generate a new API token.\\nUse IBMProvider.save_account(`APIKEY`, overwrite=True) to save the new API key, overwriting any outdated or incorrect credentials.","more_info":"https://cloud.ibm.com/apidocs/quantum-computing#error-handling"}],"trace":"beb4413b-de7e-4812-8431-4c1f20a3089c"}'

## Run SQD

In [23]:
import pyscf.tools
from pyscf import ao2mo

import collections
from functools import partial
import os
import pickle

from qiskit.primitives import BitArray
from qiskit_addon_sqd.fermion import SCIResult, diagonalize_fermionic_hamiltonian, solve_sci_batch

In [ ]:
mode_order = pickle.load(
    open(f"{circuit_dir}/{fragment}/{fragment}_mode_order_{adapt_iterations:03d}_adaptiterations.pkl", "rb")
)
qubit_order = pickle.load(
    open(f"{circuit_dir}/{fragment}/{fragment}_qubit_order_{adapt_iterations:03d}_adaptiterations.pkl", "rb")
)

In [ ]:
hamiltonian_dir = "hamiltonians"

In [ ]:
fcidump = pyscf.tools.fcidump.read(f"{hamiltonian_dir}/{fragment}.fcidump")

In [ ]:
n_orbitals = fcidump.get("NORB")
num_electrons = fcidump.get("NELEC")
ecore = fcidump.get("ECORE")
h1 = fcidump.get("H1")
h2 = fcidump.get("H2")
h2 = ao2mo.restore(1, h2, n_orbitals)

In [ ]:
energy_tol = 1e-8
occupancies_tol = 1e-8
carryover_threshold = 1e-5

In [ ]:
fname = "atp_0_be2_f4_040_adaptiterations.qasm_counts_ibm_boston_nshots_100000_2026_02_03_01-33-50"

In [ ]:
counts = pickle.load(
    open(f"results/{fragment}/{fname}", "rb")
)

In [ ]:
def transform_bitstring(bits):
    """
    Convert a given bitstring from Openfermion convention 
    (alternating alpha/beta, big endian) to Qiskit (all alpha
    then all beta, little endian).
    """

    left = [bits[i] for i in range(len(bits)) if i % 2 == 1]   # beta
    right = [bits[i] for i in range(len(bits)) if i % 2 == 0]  # alpha

    # Reverse each half
    left.reverse()
    right.reverse()

    # Concatenate
    return ''.join(left + right)

In [ ]:
measurement_outcomes = counts
permuted_outcomes = {}
for original_bitstring in measurement_outcomes.keys():
    qubit_permuted_bitstring = "".join([original_bitstring[qubit_order.index(n)] for n in range(nqubits)])
    mode_permuted_bitstring = "".join([qubit_permuted_bitstring[mode_order.index(n)] for n in range(nqubits)])

    final_permuted_bitstring = transform_bitstring(mode_permuted_bitstring)
    permuted_outcomes[final_permuted_bitstring[::]] = measurement_outcomes[original_bitstring]

bit_array = BitArray.from_counts(permuted_outcomes)
counts = bit_array.get_counts()
max_key = max(counts, key=counts.get)
print(f'Most common bitstring: {max_key} with count {counts[max_key]}')
print(f'Total number of bitstrings: {len(counts)}')
print(f"Total number of samples:", sum(counts.values()))

In [ ]:
sci_solver = partial(solve_sci_batch, spin_sq=0, max_cycle=10000)
result_history = []

def callback(results: list[SCIResult]):
    result_history.append(results)
    iteration = len(result_history)
    print(f"Iteration {iteration}")
    for i, result in enumerate(results):
        print(f"\tSubsample {i}")
        print(f"\t\tEnergy: {result.energy + ecore}")
        print(f"\t\tSubspace dimension: {np.prod(result.sci_state.amplitudes.shape)}")


result = diagonalize_fermionic_hamiltonian(
    one_body_tensor=h1,
    two_body_tensor=h2,
    bit_array=bit_array,
    samples_per_batch=500,
    norb=n_orbitals,
    nelec=(num_electrons // 2, num_electrons // 2),
    num_batches=2,
    energy_tol=energy_tol,
    occupancies_tol=occupancies_tol,
    max_iterations=100,
    sci_solver=sci_solver,
    symmetrize_spin=True,
    carryover_threshold=carryover_threshold,
    callback=callback,
)